# Esercizio 7 - Frutta con la M

In questo esercizio il sistema estrarrà una lettera a caso e chiederà all'utente di inserire un frutto con la lettera scelta, il 
frutto deve essere inserito in lingua inglese siccome usiamo wordnet per il controllo.

Abbiamo gestito l'esercizio come un gioco in cui l'utente deve battere la macchina che lo interroga.

### Approccio

Il problema di questo esercizio è far capire al sistema quando l'input è un frutto. Per risolvere questo problema ci siamo serviti di wordnet
che però è strutturato in modo che non tutti i frutti abbiamo come iperonimo diretto il synset *fruit*.

Un altro assunto molto importante da fare è che, in wordnet, tutti i frutti hanno come iperonimo *fruit*, ma non tutti gli iponimi di
fruit sono frutti.

Per guidare wordnet nella ricerca gli passiamo una lista di parole, in questo caso una lista di frutti. Dopo di che andiamo a ricercare
il synset di ogni parola e dopo di che gli iperonimi diretti. In questo modo riusciamo ad escludere alcuni iponimi di *fruit* dalla ricerca.

Dopo aver trovato gli iperonimi di partenza, andiamo a ricavare tutti i loro iponimi, e così facendo abbiamo la nostra lista di frutti.

Il problema è che in questo modo catturiamo anche delle categorie che non sono dei frutti, come ad esempio le piante. Per 
risolvere questo problema, una volta trovata la lista di synset (lista di iponimi degli iperonimi delle parole di partenza),
andiamo a rimuovere tutti i synset che non hanno come iperonimo *fruit*.

Il nostro sistema è stato studiato solo per riconoscere i frutti, ma il nostro approccio, con qualche piccola modifica, è possibilmente 
estendibile ad altre categorie.

### Imports and constant

In [1]:
from nltk.corpus import wordnet as wn
from pprint import pprint
import time
import string
import random

Questa lista rappresenta la nostra memoria, il nostro sistema andrà a cercare il synset di riferimento in wordnet basandosi sui
dati passati in input.

In [2]:
CATEGORIES = ["FRUIT", "cranberry", "apple", "peanut", "orange"]

### Methods

In [31]:
def get_synset(word):
    '''
    Retrurn the first synset of a word.
    '''
    if(len(wn.synsets(word)) > 0):
        return wn.synsets(word)[0]
    return None

def get_hypers(word):
    '''
    Return the direct hypernyms of a word.
    '''
    syn = get_synset(word)
    hypers = []
    if(syn is not None):
        hypers = syn.hypernyms()
    return hypers

def get_all_hypers(word):
    '''
    Return the direct hypernyms of a word.
    '''
    syn = get_synset(word)
    hypo_list = []
    while(syn.hypernyms() != []):
        hypo_list.append(syn.hypernyms()[0])
        syn = syn.hypernyms()[0]
        
        
def get_words_cat_hypers(category):
    hypers = []
    for word in category[1:]:
        hypers.append(get_hypers(word))
        
    hypers = [x for xs in hypers for x in xs] # list of list to list
    return hypers

def get_hypos(syn):
    '''
    Return all the hyponyms of a word.
    '''
    hypo_list = []
    if(syn is not None):
        hypo_list = list(set([get_synset(w) for s in syn.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    return hypo_list

def get_all_hypos(syns):
    '''
    Given a list of synsets return one list of all hypos of each synsets
    '''
    hypo_list = []
    for syn in syns:
        hypo_list.append(get_hypos(syn))
    return hypo_list

def clean_response(syns):
    for syn in syns:
        lch = wn.synset('fruit.n.01').lowest_common_hypernyms(syn)
        fruit = wn.synset('fruit.n.01')
        if(lch[0] != fruit):
            syns.remove(syn)
    return syns

def check_response(response, user_response, letter):
    '''
    Check if the users response start with the right letter and it's a fruit
    '''
    ok_answer = []
    for el in response:
        lemma = el.lemma_names()[0]
        if(lemma[0] == letter):
            ok_answer.append(lemma)
        
    if user_response.lower() in ok_answer:
        return True
    return False

def pick_word(syns, letter):
    '''
    Return a word (fruit) that start with the right letter
    '''
    answer = "1"
    while answer[0] != letter:
        answer = random.choice(syns).lemma_names()[0]
    return answer


### Main

In [33]:
def game():
    print("Welcome to the Game!")

    letter = random.choice(string.ascii_letters).lower()
    
    # letter = "o"
    
    print("Tell me a fruit starting with letter ...", letter.capitalize())

    user_response = input()
    
    hyper_list = get_words_cat_hypers(CATEGORIES)
    
    right_syn = get_all_hypos(hyper_list)
    right_syn = [x for xs in right_syn for x in xs] # list of list to list
    
    # Remove all the synset that are not hypos of fruit
    print(len(right_syn))
    right_syn = clean_response(right_syn)
    print(len(right_syn))
    
    if(check_response(right_syn, user_response, letter)):
        print("Good Job! The answer", user_response, "is correct!")
    else:
        print("\nWrong Answer!", "\n\nYou could say: ", pick_word(right_syn, letter))

game()

Welcome to the Game!
Tell me a fruit starting with letter ... Y
1150
629

Wrong Answer! 

You could say:  yellow_honeysuckle


### Analisi dei risultati e limiti del sistema

In questo esercizio più che negli altri abbiamo potuto toccare con mano quanto wordnet sia imprevedibile. E' indubbiamente una risorsa
molto potente, ma spesso va fatto un po' di filtraggio dei risultati ottenuti.

Idealmente in questo esercizio sarebbe bastato contollare se la parola passata in input era un iponimo di *fruit*. Questo non è stato possibile
siccome *fruit* ha tantissimi iponimi, e solo una parte sono frutti intesi come li intendiamo noi, infatti come iponimi ci sono anche collegamenti
ad altre categorie, come al mondo delle piante ad esempio.

In precendenza abbiamo detto che questo approccio è applicabile anche ad altre categoria, che in parte è vero. Però richiede una parte
di analisi di wordnet e di come è rappresentata quella determinata categoria in wordnet.

In definitiva possiamo però dire che il nostro sistema, quindi wordnet, è molto abile in questo gioco siccome riesce a trovare
anche i frutti più improbabili, sintomo del fatto che wordnet sia una risorsa molto completa a livello di termini.